8/19/23 This failed again today. I will try again in a few days.
I was able to find an API key in a Jupyter notebook that I found, and was lucky that it worked as the free API Key.
I contacted customer service, they never responded.
Their repo has not been updated much recently.
Their page that holds all of the important values that they test against your own model is currently down.
Luckily, Colab is able to run Upgini (even without the API) if the only input is "Date." I verified that this
code works in Colab, and much ran the last time I tested it before Upgini stopped working for me.

In [1]:
%pip install -Uq upgini catboost
%pip install -Uq scikit-learn
%pip install -Uq xgboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Many of these imports are important to get the notebook to connect to the research.upgini database

In [2]:
import pandas as pd
import socket
import requests
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
from upgini import SearchKey
from upgini.ads import upload_user_ads

In [3]:
df = pd.read_csv('Sales_Product_Price_by_Store.csv')

In [4]:
df["Store"] = df["Store"].astype(str)
df["Product"] = df["Product"].astype(str)

# Convert date column to datetime pandas object
df["Date"] = pd.to_datetime(df["Date"])

df.sort_values("Date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Store,Product,Date,Is_Holiday,Base Price,Price,Weekly_Units_Sold
0,1,1,2010-02-05,False,9.99,7.99,245
1,7,2,2010-02-05,False,5.99,5.99,141
2,3,3,2010-02-05,False,19.99,19.99,31
3,3,2,2010-02-05,False,5.99,5.99,117
4,8,3,2010-02-05,False,19.99,19.99,70


In [5]:
X = df[df["Date"] < "2011-01-01"]
y = df[df["Date"] >= "2012-10-26"]

Notice that I commented out to see if the API key to see if it would still run.

In [6]:
from upgini import FeaturesEnricher, SearchKey

enricher = FeaturesEnricher(
    search_keys = {
        "Date":SearchKey.DATE,
        # "key_datetime": SearchKey.DATETIME,
        # "key_phone": SearchKey.PHONE,
        # "key_email": SearchKey.EMAIL,
        # "key_hashed_email": SearchKey.HEM,
        # "key_ip_address": SearchKey.IP
    },
      keep_input = True,
     api_key = "hSux-OfcWglRfQUetAPbfCFrqcqLxj4iAq31DgZfoLY"
)

UpginiConnectionError: No connection to Upgini server https://search.upgini.com to initiate external data search, but https://pypi.org is available. Please allow connection to https://search.upgini.com from Jupyter server host 172.31.80.1 or try with Google Colab https://colab.research.google.com

The remainder of the program proves the value of Upgini; it can provide a better model fit in many circumstances. I will try again in the
next few days to see if it will work.

In [ ]:
train_features = df.drop(['Weekly_Units_Sold'], axis=1)
train_target = df.Weekly_Units_Sold
enriched_train_features = enricher.fit_transform(
    train_features,
    train_target,
    scoring = "mean_absolute_error")

In [ ]:
from upgini.metadata import CVType
enricher = FeaturesEnricher(
    search_keys={"Date": SearchKey.DATE},
    cv=CVType.time_series
)

In [ ]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
from sklearn.model_selection import train_test_split

# Find all categorical features and replace NaNs with 'NA'
cat_col_enriched = [col for col in enriched_train_features.columns if enriched_train_features[col].dtype == "O"]
enriched_train_features.loc[:, cat_col_enriched] = enriched_train_features.loc[:, cat_col_enriched].fillna("NA")

cat_col_baseline = [col for col in train_features.columns if train_features[col].dtype == "O"]
train_features.loc[:, cat_col_baseline] = train_features.loc[:, cat_col_baseline].fillna("NA")

# Train and test split for correct model evaluation
X_train, X_test, y_train, y_test, X_train_baseline, X_test_baseline = train_test_split(
    enriched_train_features,
    train_target,
    train_features,
    test_size=0.2,
    shuffle=True,
    random_state=0)

# Task-optimized Catboost estimator
model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=330,
    random_state=0,
    eval_metric="MAE",
    verbose=False,)

In [ ]:
model.fit(X_train_baseline, y_train, cat_features=cat_col_baseline)
preds = model.predict(X_test_baseline)
eval_metric(y_test.values, preds, "MAE")

In [ ]:
model.fit(X_train, y_train, cat_features=cat_col_enriched)
preds = model.predict(X_test)
eval_metric(y_test.values, preds, "MAE")